<a href="https://colab.research.google.com/github/AdeebZ-tech/Personalised-Cancer-Treatment-Optimal-Control-with-Evolutionary-Algortihm-for-Drug-Administration/blob/main/Patient_A_NSGA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q amplpy
!pip install -U pymoo
!pip install pymcdm
!pip install pandas
!pip install numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210078 sha256=6b5df7de4e43ea91468dd4fd0ac1142633f336f3175fdf5bbc4fe5d910a5dc3a
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
from amplpy import AMPL, tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.termination import get_termination
from pymoo.visualization.scatter import Scatter
from pymcdm.methods import TOPSIS
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling



In [ ]:
ampl = tools.ampl_notebook(
    modules=["coin", "highs", "gokestrel","cbc"], # modules to install
    license_uuid="1fd4b8a1-bbdd-426f-b7f9-f89a5b106b01", # license to use
    g=globals()) # instantiate AMPL object and register magics

Licensed to AMPL Community Edition License for <adeebzikri0@gmail.com>.


In [ ]:
ampl = AMPL()
ampl.eval('reset;')

In [ ]:
ampl.eval('reset;')

ampl.eval(r"""

reset;

# Terminal time tf, number n of grid, and step size h
param tf := 140;
param n := 14000;
param h := tf / n;

# Parameters
param a1 := 0.2;
param a2 := 0.3;
param a3 := 0.1;
param c1 := 1;
param c2 := 0.5;
param c3 := 1;
param c4 := 1;
param b1 := 1;
param b2 := 1;
param d1 := 0.2;
param d2 := 1.0;
param r1 := 1.5;
param r2 := 1;
param s := 0.33;
param rho := 0.01;
param alpha := 0.3;

param w1 ;
param w2 := 1-w1;
param umax;

## Initial values
param I0 := 0.15;
param N0 := 1;
param T0 := 0.2;
param M0 := 0;

#### State variables ###########

var N{i in 0..n} >= 0;
s.t. lN0: N[0] = N0;
s.t. etaN{i in 0..n}: N[i] >= 0.74;

var T{i in 0..n} >= 0;
s.t. lT0: T[0] = T0;

var I{i in 0..n} >= 0;
s.t. lI0: I[0] = I0;

var M{i in 0..n} >= 0;
s.t. lM0: M[0] = M0;

var int{i in 0..n};
s.t. lint0: int[0] = 0;



##### Control variables and constraints #########

var u{i in 0..n} >= 0, <= umax;


# Right-hand sides of ODEs

var fN{i in 0..n} = r2 * N[i] * (1 - b2 * N[i]) - c4 * T[i] * N[i] - a3 * (1 - exp(-M[i]))*N[i];

var fT{i in 0..n} = r1 * T[i] * (1 - b1 * T[i]) - c2 * I[i] * T[i] - c3 * T[i] * N[i] - a2 * (1 - exp(-M[i])) * T[i];

var fI{i in 0..n} = s + rho * I[i] * T[i] / alpha + T[i] - c1 * I[i] * T[i] - d1 * I[i] - a1 * (1 - exp(-M[i])) * I[i];

var fM{i in 0..n} = u[i] - d2*M[i];



## Cost function
var fint{i in 0..n} = w1 * T[i] + w2 * M[i];

# Objective
minimize obj: int[n];

## ODEs:  Euler
s.t. lambda_N{i in 0..n-1}: N[i+1] = N[i] + h * fN[i+1];
s.t. lambda_T{i in 0..n-1}: T[i+1] = T[i] + h * fT[i+1];
s.t. lambda_I{i in 0..n-1}: I[i+1] = I[i] + h * fI[i+1];
s.t. lambda_M{i in 0..n-1}: M[i+1] = M[i] + h * fM[i+1];

s.t. lambda_int{i in 0..n-1}: int[i+1] = int[i] + h * fint[i+1];

option abs_boundtol 1;

""")



In [ ]:
# Define the objective function
def objective_function(umax, w1):
    print(f"w1: {w1}, umax: {umax}")  # Print the values of w1 and umax

    ampl.param['umax'] = umax
    ampl.param['w1'] = w1

    ampl.option['solver'] = 'ipopt'
    ampl.eval('option ipopt_options "max_iter=2000 tol=1e-8";')
    ampl.solve(verbose=False)

    if ampl.get_value('solve_result') == 'solved':
        print("Optimal Solution Found")

        ampl.eval('display obj, T[n], M[n];')

        data_T = ampl.get_data('T')
        df1 = pd.DataFrame(data_T, columns=['index0', 'T'])
        T = df1['T'].sum()

        data_M = ampl.get_data('M')
        df2 = pd.DataFrame(data_M, columns=['index0', 'M'])
        M = df2['M'].sum()


        data_N = ampl.get_data('N')
        df3 = pd.DataFrame(data_N, columns=['index0', 'N'])
        N = df3['N'].min()

        f1 = T
        f2 = M

        g1 = 0.74 - N

        return [f1, f2, g1]
    else:
        print("Optimal Solution NOT Found")
        return [float('inf'), float('inf'), float('inf')]


In [ ]:
# Define the problem class for pymoo
class MyProblem(Problem):
    def __init__(self):
        super().__init__(n_var=2,  # specify number of decision variables
                         n_obj=2,  # specify number of objectives
                         n_ieq_constr=1,  # specify number of constraints
                         xl=np.array([0, 0]),  # specify lower bounds of decision variables
                         xu=np.array([1, 1]))  # specify upper bounds of decision variables

    def _evaluate(self, X, out, *args, **kwargs):
        F = np.empty((X.shape[0], 2))
        G = np.empty((X.shape[0], 1))
        for i in range(X.shape[0]):
            umax = X[i, 0]
            w1 = X[i, 1]
            f1, f2, g1 = objective_function(umax, w1)
            F[i, 0] = f1
            F[i, 1] = f2
            G[i, 0] = g1

        out["F"] = F
        out["G"] = G


In [ ]:
# Initialize the problem
problem = MyProblem()

# Termination criteria
termination = get_termination("n_gen", 10)

# Define algorithms to test
algorithms = [NSGA2(pop_size=50,  # A moderate population size
    n_offsprings=20,  # Generate offspring at 40% of the population size
    sampling=FloatRandomSampling(),  # Sampling for continuous variables
    crossover=SBX(prob=0.9, eta=20),  # Crossover for continuous variables
    mutation=PM(eta=20),  # Mutation for continuous variables
    eliminate_duplicates=True)  # Maintain diversity),
    #NSGA3(ref_dirs=get_reference_directions("das-dennis", 2, n_partitions=12)),
    #AGEMOEA2()
]



In [ ]:
# Perform optimization with each algorithm
results = []
for algo in algorithms:
    print(f"******** {type(algo).__name__} ********")
    res = minimize(problem,
                   algorithm=algo,
                   termination=termination,
                   seed=1,
                   save_history=True,
                   verbose=True)
    results.append(res)

    # Extract the W1 values from the final population
    w1_values = res.X[:, 1]
    print(f"w1 values: {w1_values}")

    # Calculate the W2 values
    w2_values = 1 - w1_values
    print(f"w2 values: {w2_values}")


******** NSGA2 ********
w1: 0.7203244934421581, umax: 0.417022004702574
Optimal Solution Found
obj = 1.26443
T[n] = 6.93267e-20
M[n] = 6.19471e-06

w1: 0.30233257263183977, umax: 0.00011437481734488664
Optimal Solution Found
obj = 0.530743
T[n] = 6.93209e-20
M[n] = 2.29165e-06

w1: 0.0923385947687978, umax: 0.14675589081711304
Optimal Solution Found
obj = 0.162148
T[n] = 6.932e-20
M[n] = 1.89996e-06

w1: 0.34556072704304774, umax: 0.1862602113776709
Optimal Solution Found
obj = 0.60662
T[n] = 6.93212e-20
M[n] = 2.63896e-06

w1: 0.538816734003357, umax: 0.39676747423066994
Optimal Solution Found
obj = 0.945835
T[n] = 6.93229e-20
M[n] = 3.74907e-06

w1: 0.6852195003967595, umax: 0.4191945144032948
Optimal Solution Found
obj = 1.20281
T[n] = 6.93256e-20
M[n] = 5.49901e-06

w1: 0.8781174363909454, umax: 0.20445224973151743
Optimal Solution Found
obj = 1.5414
T[n] = 6.93391e-20
M[n] = 1.56355e-05

w1: 0.6704675101784022, umax: 0.027387593197926163
Optimal Solution Found
obj = 1.17692
T[n] =

In [ ]:
from ma
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="blue")
plot.show()

SyntaxError: invalid syntax (<ipython-input-1-e6b5f145743a>, line 1)

In [ ]:
ret = [np.min(e.pop.get("F")) for e in res.history]

plt.plot(np.arange(len(ret)), ret, label="nsga2")
plt.title("Convergence")
plt.xlabel("Generation")
plt.ylabel("F")
plt.legend()
plt.show()

In [ ]:
## Apply TOPSIS

F = res.F
G = res.G

# Normalize the objective function and constraint violation values
F_norm = (F - F.min(axis=0)) / (F.max(axis=0) - F.min(axis=0))
G_norm = (G - G.min(axis=0)) / (G.max(axis=0) - G.min(axis=0))

# Calculate the weighted sum of the objective function values
weighted_sum = w1_values[:, None] * F_norm[:, 0] + w2_values[:, None] * F_norm[:, 1]

# Find the index of the best solution
best_idx = np.argmin(weighted_sum)

# Print the best solution
print(f"Best solution:")
print(f"  W1 value: {w1_values[best_idx]}")
print(f"  W2 value: {w2_values[best_idx]}")
print(f"  Objective function values: {F[best_idx]}")
print(f"  Constraint violation values: {G[best_idx]}")

# Plot the Pareto front and mark the best solution
plt.scatter([ind[0] for ind in F], [ind[1] for ind in F], color='blue', label='Pareto front')
plt.scatter(F[best_idx][0], F[best_idx][1], marker='x', color='red', s=200, label='Best solution')

plt.xlabel('F1')
plt.ylabel('F2')
plt.legend()
plt.title('Pareto Front')
plt.show()

In [ ]:
# Extract the W1 values from the final population
w1_values = res.X[:, 1]

# Calculate the W2 values
w2_values = 1 - w1_values

## Apply TOPSIS

F = res.F
G = res.G

# Normalize the objective function and constraint violation values
F_norm = (F - F.min(axis=0)) / (F.max(axis=0) - F.min(axis=0))
G_norm = (G - G.min(axis=0)) / (G.max(axis=0) - G.min(axis=0))

# Calculate the weighted sum of the objective function values
weighted_sum = w1_values[:, None] * F_norm[:, 0] + w2_values[:, None] * F_norm[:, 1]

# Find the best solution (closest to the ideal point): np.linalg.norm() to calculate the Euclidean distance
ideal_point = np.min(F_norm, axis=0)
distances = np.linalg.norm(F_norm - ideal_point[None, :], axis=1)
best_idx = np.argmin(distances)

# Print the best solution
print(f"Best solution:")
print(f"  W1 value: {w1_values[best_idx]}")
print(f"  W2 value: {w2_values[best_idx]}")
print(f"  Objective function values: {F[best_idx]}")
print(f"  Constraint violation values: {G[best_idx]}")

# Plot the Pareto front and mark the best solution
plt.figure(figsize=(8, 6))
plt.scatter(F[:, 0], F[:, 1], color='blue', label='Pareto front')
plt.scatter(F[best_idx][0], F[best_idx][1], marker='x', color='red', s=200, label='Best solution')

plt.xlabel('F1')
plt.ylabel('F2')
plt.legend()
plt.title('Pareto Front')
plt.show()

In [ ]:
from pymoo.indicators.hv import HV

# Calculate the utopian point
utopian_point = np.min(F, axis=0)
print(f"Utopian point: {utopian_point}")

# Create the Hypervolume instance and compute the hypervolume
hv = HV(ref_point=utopian_point + 0.1)
hv_value = hv.do(F_norm)
print(f"Hypervolume: {hv_value}")

In [ ]:
#Retrieve the results
data_N = ampl.get_data('N')
df1 = pd.DataFrame(data_N, columns=['index0', 'N'])

data_T = ampl.get_data('T')
df2 = pd.DataFrame(data_T, columns=['index0', 'T'])

data_I = ampl.get_data('I')
df3 = pd.DataFrame(data_I, columns=['index0', 'I'])

data_M = ampl.get_data('M')
df4 = pd.DataFrame(data_M, columns=['index0', 'M'])

data_u = ampl.get_data('u')
df5 = pd.DataFrame(data_u, columns=['index0', 'u'])

##plotting
tf = 140
N = 14000

# Time grid for plotting
tgrid = np.linspace(0, tf, N+1)

# Plotting
plt.figure(figsize=(12,8))

plt.subplot(2,3,1)
plt.plot(tgrid, df1['N'], label='N')
plt.xlabel('Time')
plt.ylabel('N')
plt.grid(True)
plt.title('N')

plt.subplot(2,3,2)
plt.plot(tgrid, df2['T'], label='T')
plt.xlabel('Time')
plt.ylabel('T')
plt.grid(True)
plt.title('T')

plt.subplot(2,3,3)
plt.plot(tgrid, df3['I'], label='I')
plt.xlabel('Time')
plt.ylabel('I')
plt.grid(True)
plt.title('I')

plt.subplot(2,3,4)
plt.plot(tgrid, df4['M'], label='M')
plt.xlabel('Time')
plt.ylabel('M')
plt.grid(True)
plt.title('M')

plt.subplot(2,3,5)
plt.plot(tgrid, df5['u'], label='u')
plt.xlabel('Time')
plt.ylabel('u')
plt.grid(True)
plt.title('control delivery')

plt.tight_layout()
plt.show()

In [ ]:
## print adjoint variables

data_lambda_N= ampl.get_data('lambda_N')
df6 = -1*pd.DataFrame(data_lambda_N, columns=['index0','lambda_N'])

data_lambda_T = ampl.get_data('lambda_T')
df7 = -1*pd.DataFrame(data_lambda_T, columns=['index0','lambda_T'])

data_lambda_I = ampl.get_data('lambda_I')
df8 = -1*pd.DataFrame(data_lambda_I, columns=['index0','lambda_I'])

data_lambda_M = ampl.get_data('lambda_M')
df9 = -1*pd.DataFrame(data_lambda_M, columns=['index0','lambda_M'])

data_phi = ampl.get_data('lambda_M')
df10 = pd.DataFrame(data_lambda_M, columns=['index0','lambda_M'])


# Time grid for plotting
tgrid1 = np.linspace(0, tf, N)

# Plotting
plt.figure(figsize=(12,8))
plt.subplot(2,3,1)
plt.plot(tgrid1, df6['lambda_N'], label='lambda_N')
plt.xlabel('Time')
plt.ylabel('lambda_N')
plt.grid(True)
plt.title('co-state N')

plt.subplot(2,3,2)
plt.plot(tgrid1, df7['lambda_T'], label='lambda_T')
plt.xlabel('Time')
plt.ylabel('lambda_T')
plt.grid(True)
plt.title('co-state T')

plt.subplot(2,3,3)
plt.plot(tgrid1, df8['lambda_I'], label='lambda_I')
plt.xlabel('Time')
plt.ylabel('lambda_I')
plt.grid(True)
plt.title('co-state I')

plt.subplot(2,3,4)
plt.plot(tgrid1, df9['lambda_M'], label='lambda_M')
plt.xlabel('Time')
plt.ylabel('lambda_M')
plt.grid(True)
plt.title('co-state M')

plt.subplot(2,3,5)
plt.plot(tgrid1, df10['lambda_M'], label='phi')
plt.xlabel('Time')
plt.ylabel('phi')
plt.grid(True)
plt.title('dH/du')

plt.tight_layout()
plt.show()
